In [7]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from langchain_openai import ChatOpenAI
from openai import OpenAI


## Creating a math tutor assistant

In [9]:
# OpenAIAssistantRunnable is broken until new pr is merged

client = OpenAI()
interpreter_assistant = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4",
    client=client,
)
output = interpreter_assistant.invoke({"content": "What's 10 - 4 raised to the 2.7"})
output

ValueError: Unsupported function

{'type': 'code_interpreter', 'function': 'function'}

Functions must be passed in as Dict, pydantic.BaseModel, or Callable. If they're a dict they must either be in OpenAI function format or valid JSON schema with top-level 'title' and 'description' keys.

In [39]:
from langchain.agents import AgentExecutor

tools = []
agent = OpenAIAssistantRunnable(assistant_id="asst_pRec50mPTEPJVrW1b1I1BIV5", as_agent=True, tools=tools)

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [40]:
from langchain_core.agents import AgentFinish


def execute_agent(agent, tools, input):
    tool_map = {tool.name: tool for tool in tools}
    response = agent.invoke(input)
    while not isinstance(response, AgentFinish):
        tool_outputs = []
        for action in response:
            tool_output = tool_map[action.tool].invoke(action.tool_input)
            print(action.tool, action.tool_input, tool_output, end="\n\n")
            tool_outputs.append(
                {"output": tool_output, "tool_call_id": action.tool_call_id}
            )
        response = agent.invoke(
            {
                "tool_outputs": tool_outputs,
                "run_id": action.run_id,
                "thread_id": action.thread_id,
            }
        )

    return response

In [41]:
response = execute_agent(agent, tools, {"content": "What's 10 - 4 raised to the 2.7"})
print(response.return_values['output'])

AttributeError: module 'openai.types.beta.threads' has no attribute 'MessageContentText'

In [3]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents.openai_assistant import OpenAIAssistantRunnable


tools = [DuckDuckGoSearchRun()]

In [4]:
agent = OpenAIAssistantRunnable.create_assistant(
    name="langchain assistant e2b tool",
    instructions="You are a personal math tutor. Write and run code to answer math questions. You can also search the internet.",
    tools=tools,
    model="gpt-4-1106-preview",
    as_agent=True,
)